# WEEK3
## Seq2Seq
机器翻译任务——将一串法语翻译成英语     
<img style="float: center;" src="course_5_pics/WEEK_3_1.PNG" width=500 height=500>     

**定义两个网络:Encoder与Decoder**    
**Encoder**:接受输入序列，输出一个向量表征这个序列(编码)     
**Decoer**:将Encoder的输出做为输入，每步输出一个值——直到输出序列的结尾。    
<img style="float: center;" src="course_5_pics/WEEK_3_2.PNG" width=500 height=500>       

语言模型与机器翻译的差别:     
- 语言模型使用是个随机输入与**随机采样**来形成一个文本序列。
<img style="float: center;" src="course_5_pics/WEEK_3_3.PNG" width=500 height=500>
- 机器翻译的输出部分接受的**不是随机输入**，而是一个**编码向量**。并且与语言模型不同的是，机器翻译追求$P(y^{<1>},y^{<2>}...y^{<T_y>}|x)$的最优值，因此在每次输出后不是采用**随机采样**，而是**集束搜索**       
<img style="float: center;" src="course_5_pics/WEEK_3_4.PNG" width=500 height=500>              

## Beam Search（集束搜索）                

1. 第一步:集束搜索在Decoder每次输出时会考虑可能性最大的$n$个单词，$n$是集束宽度($Beam \ width$)
2. 第二步:由于第一步搜索出了$n$个词语，得到第一个输出的概率$P(y^{<1>}|x)$;根据第一步$n$个输出，继续估计下一个词语，得到概率值$P(y^{<1>},y^{<2>}|x)$，这样一共会有$Beam\ width*Number\ of\ words$个词语，再根据条件概率选出其中最大的$n$个词语。        

<img style="float: center;" src="course_5_pics/WEEK_3_5.JPG" width=500 height=500>

### 改进集束搜索的技巧       

1. 集束搜索中，每个词语的概率值都是小于1，因此连乘可能导致数值下溢——解决方法:取对数   

$$P(y^{<1>},y^{<2>}...y^{<T_y>}|x) = P(y^{<1>}|x)P(y^{<2>}|x,y^{<1>})....P(y^{<T_y>}|x,y^{<1>},...y^{<T_y-1>})$$            

$$argmaxP(y^{<1>},y^{<2>}...y^{<T_y>}|x)$$    

$$\Rightarrow \underset{y}{arg\ max}\prod P(y^{<t>}|x,y^{<1>}...y^{<t-1>})$$    

$$\Rightarrow \underset{y}{arg\ max}\sum log(P(y^{<t>}|x,y^{<1>}...y^{<t-1>}))$$       

2. 集束搜索中，最优化函数倾向于更短的句子——解决方法:归一化(除以句子长度或句子长度的$\alpha(=0.7,usually)$ 次幂)      

$$\frac{1}{T_y^\alpha} P(y^{<1>},y^{<2>}...y^{<T_y>}|x)$$    


### 集束搜索的误差分析

机器翻译句子:@@jane is a pussy boy@@@@@@—— $\hat y$              
人类翻译句子:##jane is a bad boy######—— $y^*$ 

利用RNN计算两个句子的概率$P(y\hat|x)$与$P(y^* |x)$     
那么存在两种情况    
1.$P(y\hat|x)>P(y^* |x)$     
网络结构出问题    
2. $P(y\hat|x)<P(y^* |x)$    
$Beam\ Search$出问题

## BLEU   
```
翻译一个法语句子，有两个参考的人工翻译  

Reference 1: The cat is on the mat.
Reference 2:There is a cat on the mat.
```

Bleu的原理:机器翻译的结果接近人工的参考翻译结果——看生成的词是否出现在参考翻译中。    

```
percision:机器翻译中的单词出现在参考翻译中的个数/总单词个数   
modified:机器翻译中的单词出现在参考翻译中的个数/总单词个数，
      每个单词有个计分上限，比如"the"，在翻译1中出现2次，翻译2中出现1次，计分上限为2
```


### n-grams的Bleu   
```
Example: 
Reference 1: The cat is on the mat.
Reference 2: There is a cat on the mat.
MT output: The cat the cat on the mat.
         count   count_clip
the cat    2       1
cat the    1       0
cat on     1       1
on the     1       1
the mat    1       1

BLEU = 4/6
```

对于**n-grams**而言:   

$$P_n = \cfrac{\sum_{n-grams\in \hat y }Content_{clip} (n-grams)}{\sum_{n-grams\in \hat y}Content(n-grams)}$$

### Combined Bleu Scores      

$$(BP)exp(\frac{1}{4}\sum_{n=1}^4P_n),BP-brief\ penalty(简短惩罚)$$    

$$
BP =\begin{cases}  
1&,if MT_{output\ length} > reference_{output\ length}\\
exp(1 — MT_{output\ length}/reference_{output\ length})&,otherwise
\end{cases} 
$$    


## 注意力模型     
    

当人在翻译句子的时候，不会一次性翻译一页，而是一段一段翻译。    
在Encoder-Decoder模型当中，长段落会导致编码在压缩向量时产生损失，相比于短段落BLEU分值低。    

考虑下面这样的一个模型    

<img style="float: center;" src="course_5_pics/WEEK_3_6.png" width=500 height=500>    

<img style="float: center;" src="course_5_pics/WEEK_3_7.png" width=500 height=500>


## 语音辨识    
<img style="float: center;" src="course_5_pics/WEEK_3_8.png" width=500 height=500>    

## Trigger word     
不平衡数据集的解决方法——听到trigger word之后持续输出一段时间的1    

<img style="float: center;" src="course_5_pics/WEEK_3_9.png" width=500 height=500>  